# Conversão de Metadados de Produtos de Ecommerce Para Representação Vetorial com MetaProd2Vec

O objetivo deste notebook é, passo a passo, implementar a conversão dos dados em uma representação vetorial (Embeddings)
<br/>
<br/>
<br/>

##### Instalando dependências

Se já executou o comando <code>pip install -r requirements.txt</code>, sugerido no arquivo README.md,
não a necessidade de instalar novamente

In [1]:
# ######### Remova o comentário para instalar as dependências ########## #

# %pip install tensorflow tensorboard pandas plotly pandas-profiling

##### Instalando o dataset

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Software.json.gz

'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [4]:
import tensorflow as tf
import tensorboard as td
import plotly
import pandas as pd
import json
import gzip
from pandas_profiling import ProfileReport
from vespa.application import Vespa

c:\DevEnv\CESAR\Pesquisa Produtos\Coleta de dados\codigo\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Conectando ao Vespa

Referências em <https://blog.vespa.ai/run-search-engine-experiments-in-Vespa-from-python/>

In [5]:
app = Vespa(url = "https://NOSSA_APLICACAO.herokuapp.com")

In [6]:
data = []
with gzip.open('./datasets/meta_Software.json.gz') as file:
    for linha in file:
        data.append(json.loads(linha.strip()))
    
print(len(data))
print("\n\n\n")
print(data[0])

26790




{'category': [], 'tech1': '', 'description': [], 'fit': '', 'title': 'HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK CONCEPTS', 'also_buy': [], 'tech2': '', 'brand': 'HOLT. RINEHART AND WINSTON', 'feature': [], 'rank': '25,550 in Software (', 'also_view': [], 'main_cat': 'Software', 'similar_item': '', 'date': '</div>', 'price': '.a-box-inner{background-color:#fff}#alohaBuyBoxWidget .selected{background-color:#fffbf3;border-color:#e77600;box-shadow:0 0 3px rgba(228,121,17,.5)}#alohaBuyBoxWidget .contract-not-available{color:gray}#aloha-cart-popover .aloha-cart{height:auto;overflow:hidden}#aloha-cart-popover #aloha-cartInfo{float:left}#aloha-cart-popover #aloha-cart-details{float:right;margin-top:1em}#aloha-cart-popover .deviceContainer{width:160px;float:left;padding-right:10px;border-right:1px solid #ddd}#aloha-cart-popover li:last-child{border-right:0}#aloha-cart-popover .aloha-device-title{height:3em;overflow:hidden}#aloha-cart-popover .aloha-thumbnail-container{height:100px

In [7]:
df = pd.DataFrame.from_dict(data)
print(len(df))

26790


#### Tutorial de projeção na documentação

<https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin?hl=pt-br>

In [ ]:
# embeddings = MetaProd2Vec(dados)
# td.projection(embeddings)

In [8]:
# Criando o prifile
profile = ProfileReport(df, title='Relatório - Pandas Profiling', html={'style': {'full_width': True}})

In [10]:
profile

Render HTML: 100%|██████████| 1/1 [00:03<00:00,  3.82s/it]


In [ ]:
# Salvando o profile
profile.to_file(output_file="Relatorio01.html")